# Domain Adapter Built-in Demo

Transfer Learning is a general and convenient framework for transfer knowledge from pretrained model and/or source domain data to target task. Specifically,
* **Fine-tuning**: Transfer knowledge from pretrained model with the same/different network structure, which greatly speedups training without accuracy regression.
* **Domain adaption**: Transfer knowledge from source domain data without target label.

Directly applying pre-trained model into target domain cannot always work due to covariate shift and label shift, while fine-tuning is also not working due to the expensive labeling in some domains. Even if users invest resource in labeling, it will be time-consuming and delays the model deployment.

In this demo, we will introduce how to use domain adaptation to transfer knowledge in medical image semantic segmentation.

# Content

* [Domain Adapter Overview](#Domain-Adapter-Overview)
* [Getting Started](#Getting-Started)
    * [Environment Setup](#Environment-Setup)
    * [Task Description](#Task-Description)
    * [Launch Training](#Launch-Training)
    * [Launch Inference & Evaluation](#Launch-Inference-&-Evaluation)
    * [Visualization](#Visualization)

# Domain Adapter Overview

Adapter aims at reusing the transferable knowledge with the help of another labeled dataset with same learning task. That is, achieving better generalization with little labeled target dataset or achieving a competitive performance in label-free target dataset.

The following picture show the network strcture of domain adaption, which add a discriminator to users' base network, and try to differentiate the souce domain data and target domain data, hence, it can force the feature extractor to learn a generalized feature representation among domains.

![Adapter](../imgs/adapter.png)

# Getting Started

## Environment Setup

1. prepare code
    ``` bash
    git clone https://github.com/intel/e2eAIOK.git
    cd e2eAIOK
    git submodule update --init –recursive
    ```
2. pull docker image
   ```
   docker pull e2eaiok/e2eaiok-pytorch112
   ```
3. run docker
   ``` bash
   docker run -it --name model_adapter --shm-size=10g --privileged --network host \
   -v ${dataset_path}:/home/vmagent/app/data  \
   -v `pwd`:/home/vmagent/app/e2eaiok \
   -w /home/vmagent/app/e2eaiok e2eaiok/e2eaiok-pytorch112 /bin/bash 
   ```
4. Run in conda and install dependency
   ```bash
   conda activate pytorch-1.12.0
   python setup.py sdist && pip install dist/e2eAIOK-*.*.*.tar.gz
   cd /home/vmagent/app/e2eaiok/modelzoo/unet
   sh patch_unet.sh
   ```
5. Start the jupyter notebook and tensorboard service
   ``` bash
   pip install jupyter && \
   nohup jupyter notebook --notebook-dir=/home/vmagent/app/e2eaiok --ip=${hostname} --port=8899 --allow-root &
   ```
   Now you can visit demso in `http://${hostname}:8899/`.

## Task Description

* Our source domain is AMOS dataset(Download AMOS data from [here](https://amos22.grand-challenge.org/Dataset/)), which provides 500 CT and 100 MRI scans with voxel-level annotations of 15 abdominal organs, including the spleen, right kidney, left kidney, gallbladder, esophagus, liver, stomach, aorta, inferior vena cava, pancreas, right adrenal gland, left adrenal gland, duodenum, bladder, prostate/uterus.
* Our target domain is KiTS dataset(Download KiTS data from [here](https://github.com/neheller/kits19)), which provides 300 CT scans with voxel-level annotations of kidney organs and kidney tumor.
* Remember to put all your data in right places, now your files should be located at:
   - Images at: ```${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/imagesTr/```
   - Labels/Segmentations at: ```${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/labelsTr/```
   - Please refer to [here](https://github.com/MIC-DKFZ/nnUNet) to know how to put all your data in your `${dataset_path}` in right format.
* Our task is to explore reliable kidney semantic segmentation methodologies with the help of labeled AMOS dataset and unlabeled KiTS dataset, evalutaion metric is kidney dice score in target domain.

## Launch Training

- We take [3D-UNet](https://arxiv.org/abs/1606.06650) as users' base model
- We will first pre-train model in AMOS dataset, and use this pre-trained model later for prameter initialization for domain adaptation
- Then we apply domain adaptation algorithm to transfer knowledge from AMOS dataset to KiTS dataset
    - We use a DANN-like model architecture, the DANN algorithm is illustrated as follows:
    ![dann](../imgs/dann.png)
- Notice: 
    - we donot use **any label** from target domain KiTS, we only use label from source domain AMOS for training
    - *For demostration, we only train 1 epochs:*

In [9]:
! cd /home/vmagent/app/e2eaiok/modelzoo/unet && sh scripts/run_single_opt.sh 1

###############################################
For that I will be using the following source data configuration:
num_classes:  15
modalities:  {0: 'CT'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'CT')])
stages...

stage:  0
{'batch_size': 2, 'num_pool_per_axis': [4, 5, 5], 'patch_size': array([ 80, 160, 160]), 'median_patient_size_in_voxels': array([140, 264, 264]), 'current_spacing': array([3.22, 1.62, 1.62]), 'original_spacing': array([3.22, 1.62, 1.62]), 'do_dummy_2D_data_aug': False, 'pool_op_kernel_sizes': [[2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [1, 2, 2]], 'conv_kernel_sizes': [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]]}


I am using source data from this folder:  /home/vmagent/app/dataset/nnUNet_preprocessed/Task508_AMOS_kidney/nnUNetData_plans_v2.1_trgSp_kits19
###############################################
#################

2022-10-14 08:03:19.386149: train loss : 1.0404
2022-10-14 08:05:40.979750: validation loss: 0.0087
2022-10-14 08:05:40.981636: Average global foreground Dice: [0.0]
2022-10-14 08:05:40.981980: (interpret this as an estimate for the Dice of the different classes. This is not exact.)
2022-10-14 08:05:41.279021: lr index 0: 0.001982
2022-10-14 08:05:41.279208: lr index 1: 0.00991
2022-10-14 08:05:41.279391: This epoch took 3509.259070 s

2022-10-14 08:05:41.280476: saving checkpoint...
2022-10-14 08:05:41.633380: done, saving took 0.35 seconds
case_00004 (2, 80, 309, 309)
debug: mirroring True mirror_axes (0, 1, 2)
step_size: 0.5
do mirror: True
data shape: (1, 80, 309, 309)
patch size: [ 80 160 160]
steps (x, y, and z): [[0], [0, 74, 149], [0, 74, 149]]
number of tiles: 9
computing Gaussian
done
prediction done
force_separate_z: None interpolation order: 1
separate z: True lowres axis [0]
separate z, order in z is 0 order inplane is 1
2022-10-14 08:07:06.740799: finished prediction
2022

## Launch Inference & Evaluation

* We use following command for perform inference and evaluation, you can find your predictions in `${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/predict/`

* In our experiement, even without the target label data, adapter achieve 10.67x training speedup, while keep the 93% performance ratio.
    * Notice: this result is for illustration only and will depend on your machine.


In [ ]:
! cd /home/vmagent/app/e2eaiok/modelzoo/unet && sh scripts/run_predict.sh

## Visualization

- For this we would advise to use [MITK](https://www.mitk.org/wiki/The_Medical_Imaging_Interaction_Toolkit_(MITK)) which already has some great [tutorials](https://www.mitk.org/wiki/Tutorials). 
    - If you have not already downloaded it, here is the [MITK Download Link](https://www.mitk.org/wiki/Downloads)
    
- Here is a demostration of visualization result from MITK on KiTS dataset

![KiTS_visualization](../imgs/KiTS_visualization.png)